In [1]:
import json
import asyncio
import time
import httpx
import aiohttp

In [2]:
headers = {'Accept': 'Application/ld+json'}
http_top_level_nav = httpx.get('https://glossary.ecoinvent.org', headers=headers)

In [3]:
top_level_nav = json.loads(http_top_level_nav.text)

In [4]:
top_level_nav["itemListElement"][5]["item"]["url"]

'https://glossary.ecoinvent.org/intermediate-exchanges/'

In [5]:
http_intermediate_exchange_nav = httpx.get(top_level_nav["itemListElement"][5]["item"]["url"].replace("www.", ""), headers=headers)

In [6]:
intermediate_exchange_nav = json.loads(http_intermediate_exchange_nav.text)

In [7]:
intermediate_exchange_nav["numberOfItems"]

3837

In [8]:
intermediate_exchange_nav["itemListElement"][0]["item"]["@id"]

'https://glossary.ecoinvent.org/ids/5543d899-1cbd-4acf-a770-befd41102943/'

In [9]:
conn = aiohttp.TCPConnector(limit=None, ttl_dns_cache=300)
session = aiohttp.ClientSession(connector=conn)

In [10]:
async def gather_with_concurrency(n, *tasks):
    semaphore = asyncio.Semaphore(n)

    async def sem_task(task):
        async with semaphore:
            return await task

    return await asyncio.gather(*(sem_task(task) for task in tasks))

In [11]:
async def get_async(url):
    async with session.get(url["item"]["@id"], headers=headers) as response:
        return await response.read()

In [12]:
async def main():
    s = time.perf_counter()
    resps = await gather_with_concurrency(90, *map(get_async, intermediate_exchange_nav["itemListElement"]))
    data = [json.loads(resp) for resp in resps]
    elapsed = time.perf_counter() - s
    print(f"{elapsed}s")
    return data

In [13]:
all_int_ex = await main()

5.021055700010038s


In [14]:
session.close()

<coroutine object ClientSession.close at 0x000001943F224E80>

In [15]:
int_ex = dict()
for entry in all_int_ex:
    int_ex[entry["slug"]]=entry
    

### Ecoinvent API dataframe

In [30]:
import pandas as pd
df =  pd.DataFrame(int_ex).transpose()

In [31]:
df_transport = df.filter(like='transport', axis=0)

In [42]:
ecoinvent_names = df_transport["name"].tolist()
df_transport.head()

,@context,@id,@type,uuid,slug,name,cas_number,classifications,comment,unit,properties,product_information,formula,synonym
heat-transport-fluid-system-solar-thermal-parabolic-trough-50-mw,https://glossary.ecoinvent.org/ld-contexts/glo...,https://glossary.ecoinvent.org/ids/b2c93044-62...,ei:intermediate-exchange,b2c93044-6228-4950-833f-230346726d61,heat-transport-fluid-system-solar-thermal-para...,"heat transport fluid system, solar thermal par...",None,[{'@context': 'https://glossary.ecoinvent.org/...,None,{'@context': 'https://glossary.ecoinvent.org/l...,[],"This is an immobile infrastructure, representi...",None,[]
transport-freight-aircraft-long-haul,https://glossary.ecoinvent.org/ld-contexts/glo...,https://glossary.ecoinvent.org/ids/3317126a-4d...,ei:intermediate-exchange,3317126a-4dd7-4047-9ccf-163f94382c94,transport-freight-aircraft-long-haul,"transport, freight, aircraft, long haul",None,[{'@context': 'https://glossary.ecoinvent.org/...,None,{'@context': 'https://glossary.ecoinvent.org/l...,[],This is delivering the service of transportati...,None,[]
transport-freight-aircraft-medium-haul,https://glossary.ecoinvent.org/ld-contexts/glo...,https://glossary.ecoinvent.org/ids/220fe333-53...,ei:intermediate-exchange,220fe333-5329-4a8d-b012-1db429f2383b,transport-freight-aircraft-medium-haul,"transport, freight, aircraft, medium haul",None,[{'@context': 'https://glossary.ecoinvent.org/...,None,{'@context': 'https://glossary.ecoinvent.org/l...,[],This is delivering the service of transportati...,None,[]
transport-freight-aircraft-short-haul,https://glossary.ecoinvent.org/ld-contexts/glo...,https://glossary.ecoinvent.org/ids/07ad9bb5-f5...,ei:intermediate-exchange,07ad9bb5-f5d3-4e09-ac39-a6dcbb13a0ba,transport-freight-aircraft-short-haul,"transport, freight, aircraft, short haul",None,[{'@context': 'https://glossary.ecoinvent.org/...,None,{'@context': 'https://glossary.ecoinvent.org/l...,[],This is delivering the service of transportati...,None,[]
transport-freight-aircraft-unspecified,https://glossary.ecoinvent.org/ld-contexts/glo...,https://glossary.ecoinvent.org/ids/8795dc98-b6...,ei:intermediate-exchange,8795dc98-b6c7-4f09-be36-66739ea84900,transport-freight-aircraft-unspecified,"transport, freight, aircraft, unspecified",None,[{'@context': 'https://glossary.ecoinvent.org/...,None,{'@context': 'https://glossary.ecoinvent.org/l...,[],This is delivering the service of transportati...,None,[]


### Get Hestia Data

In [78]:
import requests
def get_hestia_transport_terms():
    x = requests.post('https://api.hestia.earth/search',
                      headers={'accept': 'application/json', 'Content-Type': 'application/json'},
                      json={"limit":1000,"offset":0,
                          "query": {
                              "bool": {
                                  "must": [
                                      {
                                          "match": {
                                              "termType": "Transport"
                                          }
                                      },
                                      {
                                          "match": {
                                              "@type": "Term"
                                          }
                                      }
                                  ]
                              }
                          }
                      })
    return x.json()['results']

### Hestia dataframe

In [79]:
df_hestia = pd.DataFrame(get_hestia_transport_terms())